In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.appName("Facebook_srossg").getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

IllegalArgumentException: u"Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':"

In [ ]:
from pymongo import MongoClient
client = MongoClient('compute-0-11', 27017)

db = client.FacebookChallenge_akar1
#train data
collection = db.fb_hw
contents = collection.find().limit(1000)
#test data
collection2 = db.fb_hw_test
contents2 = collection2.find().limit(1000)

In [ ]:
from bson import json_util, ObjectId
import json
rddSan = json.loads(json_util.dumps(contents))
#test data
rddSan2 = json.loads(json_util.dumps(contents2))

In [ ]:
rdd = sc.parallelize(rddSan)
#test data
rdd2 = sc.parallelize(rddSan2)

In [ ]:
schema = StructType([StructField("Body", StringType(), True), 
                     StructField("Id", IntegerType(), True), 
                     StructField("Tags", StringType(), True),
                     StructField("Title", StringType(), True),
                     StructField("_id", StringType(), True)])

#test data schema
schema2 = StructType([StructField("Body", StringType(), True), 
                     StructField("Id", IntegerType(), True), 
                     StructField("Title", StringType(), True),
                     StructField("_id", StringType(), True)])
train = sqlContext.createDataFrame(rdd, schema=schema)
test = sqlContext.createDataFrame(rdd2, schema=schema2)

In [ ]:
train = train.drop("_id")
#eliminate the duplicates
train = train.drop_duplicates(['Body','Title'])

In [ ]:
# load the relevant packages
from pyspark.sql import SQLContext, functions as sf
from pyspark.sql.types import StringType
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler,Tokenizer, RegexTokenizer,StopWordsRemover, StringIndexer, NGram, HashingTF, IDF,MinHashLSH,MinHashLSHModel,Word2Vec
from pyspark.ml.classification import DecisionTreeClassifier, OneVsRest,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer
import numpy as np
import pandas as pd
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import udf

In [ ]:
train.printSchema(), test.printSchema()

In [ ]:
import sys
import csv as csv 
from bs4 import BeautifulSoup
import os

In [ ]:
#Reference:https://github.com/alexeyza/Kaggle-Facebook3/blob/master/model/pre_process.py
def filter_html_tags(text):
    # the following tags and their content will be removed, for example <a> tag will remove any html links
    tags_to_filter = ['code','a']
    if isinstance(text, unicode):
        text = text.encode('utf8')
    soup = BeautifulSoup(text)
    for tag_to_filter in tags_to_filter:
        text_to_remove = soup.findAll(tag_to_filter)
        [tag.extract() for tag in text_to_remove]
    return soup.get_text()
filter_html_tags_udf = udf(filter_html_tags, StringType())
train2 = train.withColumn("body",filter_html_tags_udf("Body"))
test2 = test.withColumn("body",filter_html_tags_udf("Body"))

In [ ]:
#fitting the transformers
tokenize_body  = RegexTokenizer(inputCol="body",outputCol="body_tokenized", pattern="[^a-zA-Z#+]")
train1 = tokenize_body.transform(train2)
test1 = tokenize_body.transform(test2)
tokenize_Title  = RegexTokenizer(inputCol="Title",outputCol="title_tokenized",pattern="[^a-zA-Z#+]")
train1 = tokenize_Title.transform(train1)
test1 = tokenize_Title.transform(test1)
tokenize_tags  = Tokenizer(inputCol="Tags",outputCol="tags_tokenized")
train1 = tokenize_tags.transform(train1)
remover_body_stop = StopWordsRemover(inputCol="body_tokenized",outputCol="body_stopremove")
train1 = remover_body_stop.transform(train1)
test1 = remover_body_stop.transform(test1)
remover_title_stop = StopWordsRemover(inputCol="title_tokenized",outputCol="title_stopremove")
train1 = remover_title_stop.transform(train1)
test1 = remover_title_stop.transform(test1)

In [ ]:
#get the unique tags in the train data
tags =()
for word in train1.select("tags_tokenized").take(train1.count()):
    if word not in tags:
        tags += word
j =1
flat_taglist = []
for sublist in tags:
    for item in sublist:
        flat_taglist.append(item)
        j+=1
unique_tags = list(set(flat_taglist))    

In [ ]:
#get the count corresponding to the unique tags
tags_count={}
for i in flat_taglist:
    tags_count[i]=flat_taglist.count(i)


In [ ]:
#Average tags per sentence(Based on 10000 rows)
j/train1.count()

In [ ]:
#tried stemming the tokens but it does not impact our keywords infact it adds noise to the data with 
#wrong stemming like "consider" becoming'conside'

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer

stemmer = SnowballStemmer("english")
train3 = train1.toPandas()
train3['body_stopremove'] = train3["body_stopremove"].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
train1_renamed = train1.withColumnRenamed("tags_tokenized","label")

In [ ]:
#Combining body and title as per the probability of 1:2 (Giving more importance to words in Title than body)
from itertools import chain
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

def concat(type):
    def concat_(*args):
        return list(chain(*args))
    return udf(concat_, ArrayType(type))


concat_string_arrays = concat(StringType())

#training data
train1_concat= train1.select(concat_string_arrays(col("body_stopremove"), col("title_stopremove")),"ID","tags_tokenized","title_stopremove")
train1_concat = train1_concat.withColumnRenamed("concat_(body_stopremove, title_stopremove)", "body_title")
train2_concat= train1_concat.select(concat_string_arrays(col("body_title"), col("title_stopremove")),"ID","tags_tokenized","title_stopremove")
train2_concat = train2_concat.withColumnRenamed("concat_(body_title, title_stopremove)", "body_title")

#test data
test1_concat= test1.select(concat_string_arrays(col("body_stopremove"), col("title_stopremove")),"ID","title_stopremove")
test1_concat = test1_concat.withColumnRenamed("concat_(body_stopremove, title_stopremove)", "body_title")
test2_concat= test1_concat.select(concat_string_arrays(col("body_title"), col("title_stopremove")),"ID","title_stopremove")
test2_concat = test2_concat.withColumnRenamed("concat_(body_title, title_stopremove)", "body_title")

In [ ]:
#converting data to pandas
train1_pd = train2_concat.toPandas()
train1_pd= train1_pd.set_index('ID')

test1_pd = test2_concat.toPandas()
test1_pd= test1_pd.set_index('ID')

In [ ]:
#Reference:https://stackoverflow.com/questions/10526579/use-scikit-learn-to-classify-into-multiple-categories#comment59257073_19172087

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn import tree

X_train = np.array(train1_pd["body_title"])
y_train_text = np.array(train1_pd["tags_tokenized"])

X_test = np.array(test1_pd["body_title"])
X_test2 = np.array(train1_pd["body_title"])

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_train_text)

#fitting LinearSVC model
classifier = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda doc: doc,lowercase=False)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, Y)
predicted_test = classifier.predict(X_test)
#tested on train data data to see the fit
predicted_train = classifier.predict(X_test2)
all_labels = mlb.inverse_transform(predicted)
f1_score(Y, predicted, average='micro')
#printing the current tags and predicted tags
for item, labels in zip(y_train_text, all_labels):
    print(item,labels)
    
#F1score  :0.99966386554621844
print("testdata_tested")
print(f1_score(Y, predicted_test, average='micro'))
print("traindata_tested")
print(f1_score(Y, predicted_train, average='micro'))

In [ ]:
#Fitting Logistic Regression
classifier_lr = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda doc: doc,lowercase=False)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LogisticRegression()))])

classifier_lr.fit(X_train, Y)
predicted_test = classifier.predict(X_test)
#tested on train data data to see the fit
predicted_train = classifier.predict(X_test2)
all_labels = mlb.inverse_transform(predicted)

print("testdata_tested")
print(f1_score(Y, predicted_test, average='micro'))
print("traindata_tested")
print(f1_score(Y, predicted_train, average='micro'))

In [ ]:
#Fitting Decision tree
classifier_dt = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=lambda doc: doc,lowercase=False)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(tree.DecisionTreeClassifier()))])

classifier_dt.fit(X_train, Y)
predicted_test = classifier.predict(X_test)
#tested on train data data to see the fit
predicted_train = classifier.predict(X_test2)
all_labels = mlb.inverse_transform(predicted)

print("testdata_tested")
print(f1_score(Y, predicted_test, average='micro'))
print("traindata_tested")
print(f1_score(Y, predicted_train, average='micro'))

